In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.preprocessing as sp
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
import xgboost as xgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV, LinearRegression, BayesianRidge

# Ignore all warnings
import warnings  
warnings.simplefilter("ignore")

**CatBoost Regressor** is a gradient boosting algorithm **built on decision trees**, specifically optimized for handling numerical and categorical data in regression tasks. Unlike many other boosting frameworks, CatBoost uses an innovative technique called *ordered boosting*, which prevents target leakage during training, and *efficient encoding* of categorical variables via target statistics without manual preprocessing. It supports GPU acceleration, symmetrical tree structures for faster inference, and strong regularization capabilities to minimize overfitting. These features make CatBoost Regressor highly effective in modeling complex, non-linear relationships with minimal tuning, while maintaining high accuracy and stability across diverse datasets.

**Ridge Regression**, available in `sklearn.linear_model` as `Ridge`, is a linear regression model enhanced with L2 regularization to address issues like overfitting and multicollinearity. Unlike standard linear regression, Ridge minimizes a loss function that combines the sum of squared residuals with a penalty term proportional to the square of the model coefficients. This penalty, known as L2 regularization, constrains the magnitude of the coefficients by adding $\alpha \| \beta \|^2_2$ to the cost function, where $\alpha$ is the regularization strength. The effect of L2 regularization is to shrink large weights, reducing model variance without eliminating any variables entirely. This leads to more stable and generalizable predictions, especially in datasets with highly correlated or numerous features. Additional tools like `RidgeCV` allow automatic selection of the best regularization parameter using cross-validation. Compared to `LinearRegression`, which uses no regularization, or `BayesianRidge`, which adds probabilistic interpretation, Ridge offers a controlled and computationally efficient approach for regularized linear modeling.





In [2]:
# define RMSLE
def rmsle_score(y, preds):
    y = np.maximum(0, y)
    preds = np.maximum(0, preds)
    return np.sqrt(np.mean((np.log1p(preds) - np.log1p(y)) ** 2))

In [3]:
SEED = 42          # Set random seed for reproducibility of results
N_SPLITS = 5       # Number of folds for cross-validation (5-fold CV)
N_REPEATS = 3      # Number of times to repeat cross-validation with different splits


In [4]:
train_df=pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_df = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [5]:
print("----Train data----")
print(train_df.isnull().sum())
print("="*20)
print("----Test data----")
print(test_df.isnull().sum())

----Train data----
id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64
----Test data----
id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64


In [6]:
# Add new features
def add_new_features(df):
    # # BMI
    # df['BMI'] = df['Weight'] / ((df['Height'] / 100) ** 2)
    # # Duration / Heart Rate
    # df["Duration_per_HeartRate"] = df["Duration"] / (df["Heart_Rate"] + 1e-5)
    # # Intensity
    # df['Intensity'] = df["Heart_Rate"] / (df["Duration"] + 1e-5)
    # # Other
    # df["Duration_x_HeartRate"] = df["Duration"] * df["Heart_Rate"]
    df['Calories_Burned'] = np.where(
        df['Sex'] == 'male',
        (-55.0969 + (0.6309 * df['Heart_Rate']) + (0.1988 * df['Weight']) + (0.2017 * df['Age'])) / 4.184 * df['Duration'],
        (-20.4022 + (0.4472 * df['Heart_Rate']) - (0.1263 * df['Weight']) + (0.074 * df['Age'])) / 4.184 * df['Duration']
    )
    return df

train_df = add_new_features(train_df)
test_df = add_new_features(test_df)

In [7]:
# create Duration class column
bins = list(np.arange(1, 40, 5))
labels = [f'{b}-{b+4}' for b in bins[:-1]]

train_df['Duration_class'] = pd.cut(train_df['Duration'], bins=bins, labels=labels, right=False)
test_df['Duration_class'] = pd.cut(test_df['Duration'], bins=bins, labels=labels, right=False)


This code creates a **new categorical column** called `Duration_class` in both the training and testing DataFrames by grouping the continuous `Duration` values into intervals or “bins.” The bins are defined from 1 to 40 in steps of 5 (e.g., 1-5, 6-10, etc.), and each `Duration` value is assigned a label corresponding to the interval it falls into. Using `pd.cut` with `right=False` means each bin includes the left edge but excludes the right edge. This transformation converts the numeric `Duration` into meaningful categories, which can help models capture patterns related to different duration ranges more easily.


In [8]:
# create age class column
bins = list(np.arange(1, 90, 5))
labels = [f'{b}-{b+4}' for b in bins[:-1]]

train_df['age_class'] = pd.cut(train_df['Age'], bins=bins, labels=labels, right=False)
test_df['age_class'] = pd.cut(test_df['Age'], bins=bins, labels=labels, right=False)

In [9]:
# target encoding
# groubby --> Sex, Age_class, Duration_class
group_encod = train_df.groupby(['Sex', 'age_class', 'Duration_class'])['Calories'].median().reset_index()
group_encod.rename(columns={'Calories': 'Calories_encoded'}, inplace=True)

train_df = train_df.merge(group_encod, on=['Sex', 'age_class', 'Duration_class'], how='left')
test_df = test_df.merge(group_encod, on=['Sex', 'age_class', 'Duration_class'], how='left')

This code performs **target encoding** by grouping the training data based on the categories `Sex`, `age_class`, and `Duration_class`. For each group, it calculates the **median** of the target variable `Calories` and saves it as a new value called `Calories_encoded`. Then, it merges this encoded information back into both the training and testing DataFrames, adding a new column `Calories_encoded` that contains the median calorie value for each group. This way, the model gets additional useful information about how calories relate to combinations of sex, age group, and duration category.


In [10]:
train_df['Sex'] = train_df['Sex'].map({'male': 1, 'female': 0}).astype("float32")
test_df['Sex'] = test_df['Sex'].map({'male': 1, 'female': 0}).astype('float32')

In [11]:
from sklearn.preprocessing import OneHotEncoder  # Import OneHotEncoder for converting categorical variables to binary columns

cat_cols = ['Duration_class', 'age_class']  # Specify categorical columns to encode

encoder = OneHotEncoder(sparse=False, drop=None, handle_unknown='ignore')  
# Create encoder instance:
# sparse=False returns a dense array instead of sparse matrix
# drop=None means no category is dropped (all encoded)
# handle_unknown='ignore' avoids errors if new categories appear in test data

# train data
encoded_train = encoder.fit_transform(train_df[cat_cols])  
# Fit encoder on training data and transform categorical columns into one-hot encoded arrays

encoded_train_df = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out(cat_cols))  
# Convert encoded numpy array into a DataFrame with proper column names

train_df = pd.concat([train_df.drop(columns=cat_cols), encoded_train_df], axis=1)  
# Drop original categorical columns from train_df and add the new one-hot encoded columns

# test data
encoded_test = encoder.transform(test_df[cat_cols])  
# Transform test data using the already fitted encoder (no fitting on test data)

encoded_test_df = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out(cat_cols))  
# Convert encoded test data array into a DataFrame with proper column names

test_df = pd.concat([test_df.drop(columns=cat_cols), encoded_test_df], axis=1)  
# Drop original categorical columns from test_df and add the one-hot encoded columns


In [12]:
X = train_df.drop(columns=["id", "Calories"])
y = train_df["Calories"]

X_test = test_df.drop(columns=["id"])

In [13]:
# Hyperparameters
xgb_params =  {
    "objective": "reg:squarederror",
    "eval_metric": "rmse",
    "tree_method": "gpu_hist",
    'learning_rate': 0.02, 
    'max_depth': 10, 
    'subsample': 0.8, 
    'colsample_bytree': 0.8, 
    "random_state": SEED
}

cat_params = {
    "loss_function": "RMSE",
    "learning_rate": 0.03,
    "depth": 10,
    "l2_leaf_reg": 3.0,
    "bootstrap_type": "Bayesian",
    "bagging_temperature": 1.0, 
    "random_seed": SEED,
    "verbose": 0,
    "task_type": "GPU"
}

lgb_params = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.02,
    "n_estimators": 3000, 
    "num_leaves": 20,  
    "max_depth": 8, 
    "min_child_samples": 20, 
    "min_split_gain": 0.01,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "reg_alpha": 1.0, 
    "reg_lambda": 1.0,
    "random_state": SEED,
    "verbosity": -1,
    "feature_fraction": 0.7,
    "force_col_wise": True
}


The dictionary `xgb_params` sets the hyperparameters for an XGBoost model. The `"objective": "reg:squarederror"` tells the model it’s solving a regression problem using squared error as the loss function. `"eval_metric": "rmse"` specifies that the model’s performance will be evaluated using Root Mean Squared Error. `"tree_method": "gpu_hist"` means the model will use a fast histogram-based algorithm that runs on the GPU for faster training; this method speeds up split finding by grouping continuous features into discrete bins. The `'learning_rate': 0.02` controls how much the model updates at each step, with a smaller value making learning slower but more precise. `'max_depth': 10` limits how deep each decision tree can grow, controlling model complexity. `'subsample': 0.8` defines the fraction (80%) of the training samples randomly selected for building each tree, which helps prevent overfitting by introducing randomness. `'colsample_bytree': 0.8` sets the fraction (80%) of features (columns) randomly chosen for constructing each tree, adding further randomness to reduce overfitting. Finally, `"random_state": SEED` ensures the results are reproducible by fixing the random number generator’s seed.


The `cat_params` dictionary contains hyperparameters for training a CatBoost regression model. The `"loss_function": "RMSE"` indicates that the model is optimized to minimize the Root Mean Squared Error, a common measure of prediction accuracy for regression tasks. `"learning_rate": 0.03` controls how much the model updates its predictions in each iteration—lower values mean slower but more stable learning. `"depth": 10` specifies the maximum depth of each decision tree, influencing how complex each tree can be. `"l2_leaf_reg": 3.0` applies L2 regularization to prevent overfitting by penalizing large weights in the leaves of the trees; L2 regularization works by adding a penalty equal to the square of the magnitude of coefficients, encouraging the model to prefer smaller, more generalizable values. `"bootstrap_type": "Bayesian"` tells the model to use Bayesian bootstrapping, which adds randomness in a controlled, probabilistic way based on prior uncertainty in the data, helping to reduce overfitting and improve robustness. The `"bagging_temperature": 1.0` is a parameter specific to Bayesian bootstrapping; it controls how uniform or aggressive the sampling is—higher values increase randomness in sample weights, allowing the model to focus more on uncertain or mispredicted data points. `"random_seed": SEED` ensures that results are reproducible. `"verbose": 0` turns off detailed output during training to keep logs clean, and `"task_type": "GPU"` enables GPU acceleration for faster training, which is especially useful when working with large datasets or deep trees.



The `lgb_params` dictionary defines the hyperparameters used to configure a LightGBM regression model. The `"objective": "regression"` specifies that the task is to predict continuous numerical values. `"metric": "rmse"` sets the evaluation metric to Root Mean Squared Error, which measures the average difference between predicted and actual values. `"learning_rate": 0.02` controls how much the model updates at each step—a smaller value makes learning slower but can improve accuracy. `"n_estimators": 3000` means the model can build up to 3000 trees, allowing it to learn complex patterns if needed. `"num_leaves": 20` sets the maximum number of leaf nodes per tree; more leaves allow capturing more detail but increase the risk of overfitting. `"max_depth": 8` limits how deep each tree can grow to control model complexity. `"min_child_samples": 20` ensures that each leaf has at least 20 data points, which helps reduce overfitting. `"min_split_gain": 0.01` requires a minimum improvement in the loss function before a split is made, preventing unnecessary splits.

The `"subsample": 0.8` parameter means 80% of the data will be randomly sampled to build each tree, helping prevent overfitting. Similarly, `"colsample_bytree": 0.8` means that only 80% of the features will be used for each tree, encouraging model diversity. `"reg_alpha": 1.0` and `"reg_lambda": 1.0` apply L1 and L2 regularization, respectively. Regularization prevents overfitting by penalizing overly complex models—L1 can zero out some parameters, and L2 shrinks them gradually. `"random_state": SEED` ensures consistent results across runs. `"verbosity": -1` turns off logs for cleaner output. `"feature_fraction": 0.7` limits the fraction of features used at each iteration, similar to `colsample_bytree`.

Lastly, `"force_col_wise": True` enables **column-wise histogram construction**, which means LightGBM scans and processes one feature (column) at a time to build histograms used in split decisions. This method is more memory-efficient and faster than row-wise methods, especially when dealing with datasets that have many columns.



In [14]:
# Set up repeated K-Fold cross-validation with N_SPLITS splits, repeated N_REPEATS times
rkf = RepeatedKFold(n_splits=N_SPLITS, n_repeats=N_REPEATS, random_state=SEED)

# Initialize arrays to store out-of-fold (OOF) predictions for each model
# OOF predictions are made on validation data during cross-validation and help estimate model performance on unseen data
oof_preds_xgb = np.zeros(len(X))
oof_preds_cat = np.zeros(len(X))
oof_preds_lgb = np.zeros(len(X))

# Initialize lists to collect predictions on the test set for each model
test_preds_xgb = []
test_preds_cat = []
test_preds_lgb = []

for fold, (train_idx, val_idx) in enumerate(rkf.split(X)):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    # Transform the target variable using log1p
    y_train_log = np.log1p(y_train)
    y_val_log = np.log1p(y_val)

    # XGBoost

    # Convert training and validation data into DMatrix format (optimized format for XGBoost)
    dtrain = xgb.DMatrix(X_train, label=y_train_log)
    dval = xgb.DMatrix(X_val, label=y_val_log)
    
    # Train the XGBoost model with defined parameters
    model_xgb = xgb.train(
    params=xgb_params,               # Hyperparameters for the model
    dtrain=dtrain,                   # Training data
    num_boost_round=3000,            # Maximum number of boosting rounds
    evals=[(dtrain, "train"), (dval, "valid")],  # Show evaluation metrics for train and validation
    early_stopping_rounds=100,       # Stop training early if no improvement in 100 rounds
    verbose_eval=0                   # Don't print output during training
    )

    # Make predictions on validation set using the best iteration (based on early stopping)
    xgb_score = model_xgb.predict(dval, iteration_range=(0, model_xgb.best_iteration))
        
    # Save predictions in the correct places (out-of-fold predictions)
    oof_preds_xgb[val_idx] = xgb_score
        
    # If log1p was used, we might need to reverse it with expm1 to get actual values
    # oof_preds_xgb[val_idx] = np.expm1(xgb_score)
        
    # Prepare test data for prediction
    dtest = xgb.DMatrix(X_test)

    # Predict on test set using the trained model
    xgb_pred = model_xgb.predict(dtest, iteration_range=(0, model_xgb.best_iteration))
    
    # Save test predictions for this fold
    test_preds_xgb.append(xgb_pred)
    
    # Again, if log1p was used earlier, we may reverse it here
    # test_preds_xgb.append(np.expm1(xgb_pred))

    # CatBoost
    model_cat = CatBoostRegressor(**cat_params)
    model_cat.fit(X_train, y_train_log, verbose=False)
    cat_score = model_cat.predict(X_val)
    oof_preds_cat[val_idx] = cat_score
    # oof_preds_cat[val_idx] = np.expm1(cat_score)

    cat_pred = model_cat.predict(X_test)
    test_preds_cat.append(cat_pred)
    # test_preds_cat.append(np.expm1(cat_pred))

    
    # LGBM
    model_lgb = LGBMRegressor(**lgb_params)
    model_lgb.fit(X_train, y_train_log)
    lgb_score = model_lgb.predict(X_val)
    oof_preds_lgb[val_idx] = lgb_score
    # oof_preds_lgb[val_idx] = np.expm1(lgb_score)

    lgb_pred = model_cat.predict(X_test)
    test_preds_lgb.append(lgb_pred)
    # test_preds_lgb.append(np.expm1(lgb_pred))

    # Calculate the score by comparing with y_val before the log1p transformation
    avg_score = (np.expm1(xgb_score) + np.expm1(cat_score) + np.expm1(lgb_score)) / 3
    print(f'===============Fold: {fold+1} Average RMSLE score: {np.mean(rmsle_score(y_val, avg_score)):.5f}') 
    print(f'-------------------->  XGBoost RMSLE score: {np.mean(rmsle_score(y_val, np.expm1(xgb_score))):.5f}') 
    print(f'--------------------> CatBoost RMSLE score: {np.mean(rmsle_score(y_val, np.expm1(cat_score))):.5f}') 
    print(f'--------------------> LightGBM RMSLE score: {np.mean(rmsle_score(y_val, np.expm1(lgb_score))):.5f}') 
    print('')

    # Creating train and val data for stacking
    stacked_train = np.vstack([oof_preds_xgb, oof_preds_cat, oof_preds_lgb]).T
    stacked_test = np.vstack([np.mean(test_preds_xgb, axis=0),
                              np.mean(test_preds_cat, axis=0),
                              np.mean(test_preds_lgb, axis=0)
                             ]).T
       

    

===============Fold: 1 Average RMSLE score: 0.05924
-------------------->  XGBoost RMSLE score: 0.05992
--------------------> CatBoost RMSLE score: 0.05928
--------------------> LightGBM RMSLE score: 0.06003

===============Fold: 2 Average RMSLE score: 0.05973
-------------------->  XGBoost RMSLE score: 0.06009
--------------------> CatBoost RMSLE score: 0.05989
--------------------> LightGBM RMSLE score: 0.06063

===============Fold: 3 Average RMSLE score: 0.05902
-------------------->  XGBoost RMSLE score: 0.05946
--------------------> CatBoost RMSLE score: 0.05933
--------------------> LightGBM RMSLE score: 0.05992

===============Fold: 4 Average RMSLE score: 0.05957
-------------------->  XGBoost RMSLE score: 0.05972
--------------------> CatBoost RMSLE score: 0.05975
--------------------> LightGBM RMSLE score: 0.06062

===============Fold: 5 Average RMSLE score: 0.05893
-------------------->  XGBoost RMSLE score: 0.05932
--------------------> CatBoost RMSLE score: 0.05917
--------

This above code combines the predictions from three different models—XGBoost, CatBoost, and LightGBM—by stacking their out-of-fold predictions vertically and then transposing the result to create a new training dataset (`stacked_train`). It does the same with the averaged test predictions to form `stacked_test`. This process, called **stacking, involves using the predictions of multiple models as input features for a new model, which can improve overall accuracy by learning from the strengths of each individual model.**


In [16]:
# Prediction - Using multiple meta models

y_log = np.log1p(y)

# Ridge
ridge = RidgeCV(alphas=[0.1, 1.0, 10.0, 50.0, 100.0], cv=5)
ridge.fit(stacked_train, y_log)
ridge_preds = ridge.predict(stacked_test)

#LinearRegression
lr = LinearRegression()
lr.fit(stacked_train, y_log)
lr_preds = lr.predict(stacked_test)

# BayesianRidge
bayesian_ridge = BayesianRidge(n_iter=500, tol=1e-3, alpha_1=1e-6, alpha_2=1e-6, lambda_1=1e-6, lambda_2=1e-6)
bayesian_ridge.fit(stacked_train, y_log)
bayes_preds = bayesian_ridge.predict(stacked_test)

# Averaging
final_preds = (ridge_preds + lr_preds + bayes_preds) / 3
final_preds = np.expm1(final_preds)


This code segment above details the final prediction phase of a machine learning workflow, employing a meta-modeling or stacking approach where predictions from multiple models are combined. Initially, the target variable y is transformed using np.log1p(y) (which computes log(1+x)), likely to handle skewed data or improve model performance by making the distribution more normal; this means all subsequent model training occurs on this log-transformed target. Three different meta-models are then trained on stacked_train, which represents the out-of-fold predictions from a previous layer of base models. First, a RidgeCV model is fitted, automatically selecting the optimal alpha (regularization strength) from a predefined list ([0.1, 1.0, 10.0, 50.0, 100.0]) through 5-fold cross-validation. Second, a standard LinearRegression model is trained, providing a straightforward linear combination of the stacked features. Third, a BayesianRidge model is fit with specific regularization parameters (n_iter, tol, alpha_1, alpha_2, lambda_1, lambda_2), which uses a probabilistic approach to estimate the coefficients and the regularization terms. After training, each of these three meta-models generates predictions on the stacked_test dataset. Finally, the **final_preds are calculated by averaging the predictions from the RidgeCV, LinearRegression, and BayesianRidge models**. This ensemble technique, simple averaging, aims to leverage the strengths of each meta-model and reduce the variance of the overall prediction. The very last step, np.expm1(final_preds), reverses the initial log transformation (e 
x
 −1) to convert the predictions back to their original scale, making them interpretable and directly comparable to the untransformed target variable.

In [17]:
submission = pd.DataFrame({
    'id': test_df['id'],
    'Calories': final_preds
})

# Save
submission.to_csv('submission.csv', index=False)

In [18]:
submission

,id,Calories
0,750000,27.194003
1,750001,107.511334
2,750002,87.539132
3,750003,125.150482
4,750004,76.507740
...,...,...
249995,999995,26.158479
249996,999996,9.531712
249997,999997,73.634638
249998,999998,167.652428


The overall effect of this intricate machine learning workflow is to significantly enhance prediction accuracy and robustness for a regression task by leveraging the strengths of multiple diverse models through a sophisticated **stacking ensemble method**.

The process begins with **data preparation**, specifically applying a `np.log1p` transformation to the target variable `y`. This is **crucial for handling potentially skewed target distributions**, common in real-world regression problems. By transforming `y` to $\log(1+y)$, the distribution often becomes more symmetrical and normal-like, making it easier for models to learn patterns and **reduce the impact of outliers**.

Next, the workflow implements a **two-layer stacking architecture**. The **first layer, consisting of "base models" (XGBoost, CatBoost, and LightGBM)**, generates **"out-of-fold predictions."** This is a **critical technique to prevent data leakage**: each base model predicts on data it *did not* see during its own training. This ensures the "meta-features" derived from these predictions are **unbiased and truly represent generalization ability**. These out-of-fold predictions for the training set are then combined (stacked) to form `stacked_train`, and similarly, averaged test predictions from the base models form `stacked_test`.

The **second layer, the "meta-modeling" phase**, then trains simpler models (**`RidgeCV`**, **`LinearRegression`**, and **`BayesianRidge`**) on these `stacked_train` features. These meta-models learn how to **optimally combine the predictions of the base models**, recognizing their individual strengths and weaknesses. For instance, `RidgeCV` automatically tunes its regularization, `LinearRegression` provides a straightforward blend, and `BayesianRidge` offers a probabilistic, more robust combination.

Finally, the predictions from these three meta-models are **simply averaged** to produce `final_preds`. This final averaging step acts as an **additional ensemble layer**, further smoothing out individual model biases and **reducing variance**, leading to a more stable and often more accurate final prediction. The very last step, `np.expm1`, is **essential to revert the initial log transformation** ($e^x - 1$), returning the predictions to their original scale, making them directly comparable to the true values and easily interpretable.

---

In summary, this process creates a **powerful predictive system** by:
* **Preprocessing the target variable** for better model learning.
* **Generating diverse "meta-features"** from robust, out-of-fold predictions of strong base models (XGBoost, CatBoost, LightGBM).
* **Learning intelligent combinations** of these meta-features using different linear meta-models.
* **Ensembling the meta-model predictions** for increased stability and accuracy.
* **Transforming back** the final predictions to their original, interpretable scale.

This multi-stage approach, particularly the use of **stacking with out-of-fold predictions**, is a highly effective strategy in competitive machine learning for achieving state-of-the-art performance by capitalizing on the **collective intelligence of multiple diverse algorithms**.